In [7]:
import sqlite3
import threading
import queue

class SQLiteConnectionPool:
    _instance = None
    _lock = threading.Lock()

    def __new__(cls, db_path, pool_size=5):
        if not cls._instance:
            with cls._lock:
                if not cls._instance:
                    cls._instance = super().__new__(cls)
                    cls._instance._init_pool(db_path, pool_size)
        return cls._instance

    def _init_pool(self, db_path, pool_size):
        self.db_path = db_path
        self.pool = queue.Queue(maxsize=pool_size)
        # 预创建连接（只读模式）
        uri = f"file:{db_path}?mode=ro"
        for _ in range(pool_size):
            conn = sqlite3.connect(uri, uri=True, check_same_thread=False)
            self.pool.put(conn)

    def get_conn(self):
        return self.pool.get()

    def release_conn(self, conn):
        self.pool.put(conn)


class ConfigLoader:
    def __init__(self, db_path='../config/config.db', pool_size=5):
        self.pool = SQLiteConnectionPool(db_path, pool_size)

    def get_apis_and_provider_by_model_value(self, model_value):
        conn = self.pool.get_conn()
        try:
            cursor = conn.cursor()
            cursor.execute("SELECT id, provider_id FROM models WHERE model_value=?", (model_value,))
            row = cursor.fetchone()
            if not row:
                return None

            model_id, provider_id = row
            cursor.execute("SELECT name FROM model_providers WHERE id=?", (provider_id,))
            provider_row = cursor.fetchone()
            provider_name = provider_row[0] if provider_row else None

            cursor.execute("SELECT base_url, api_key FROM model_apis WHERE model_id=? AND enable=1", (model_id,))
            apis = cursor.fetchall()
            api_list = [{"base_url": base_url, "api_key": api_key} for base_url, api_key in apis]

            return {
                "provider_name": provider_name,
                "apis": api_list
            }
        except Exception as e:
            print(f"数据库查询异常: {e}")
            return None
        finally:
            self.pool.release_conn(conn)

    def get_all_models(self):
        conn = self.pool.get_conn()
        try:
            cursor = conn.cursor()
            cursor.execute("""
                SELECT m.model_value, m.model_name, p.name as provider_name
                FROM models m
                LEFT JOIN model_providers p ON m.provider_id = p.id
            """)
            rows = cursor.fetchall()
            models = []
            for model_value, model_name, provider_name in rows:
                models.append({
                    "model_value": model_value,
                    "model_name": model_name,
                    "provider_name": provider_name
                })
            return models
        except Exception as e:
            print(f"数据库查询异常: {e}")
            return []
        finally:
            self.pool.release_conn(conn)


if __name__ == "__main__":
    loader = ConfigLoader()
    print(loader.get_apis_and_provider_by_model_value("ernie-lite-8k"))
    print(loader.get_all_models())


{'provider_name': 'baidu', 'apis': [{'base_url': 'https://qianfan.baidubce.com/v2', 'api_key': 'bce-v3/ALTAK-4KyFUYBYZMuf8s3bxg2nO/5852d9dd9a49dccc9234f0907a1b53645456469c'}]}
[{'model_value': 'GLM-4.5-Flash', 'model_name': 'GLM-4.5-Flash', 'provider_name': 'zhipu'}, {'model_value': 'GLM-4-Flash-250414', 'model_name': 'GLM-4-Flash', 'provider_name': 'zhipu'}, {'model_value': 'GLM-Z1-Flash', 'model_name': 'GLM-Z1-Flash', 'provider_name': 'zhipu'}, {'model_value': 'GLM-4.1V-Thinking-Flash', 'model_name': 'GLM-4.1V-Thinking-Flash', 'provider_name': 'zhipu'}, {'model_value': 'GLM-4V-Flash', 'model_name': 'GLM-4V-Flash', 'provider_name': 'zhipu'}, {'model_value': 'Cogview-3-Flash', 'model_name': 'Cogview-3-Flash', 'provider_name': 'zhipu'}, {'model_value': 'CogVideoX-Flash', 'model_name': 'CogVideoX-Flash', 'provider_name': 'zhipu'}, {'model_value': 'Lite', 'model_name': 'Spark-Lite', 'provider_name': 'xunfei'}, {'model_value': 'ernie-speed-128k', 'model_name': 'ERNIE-Speed-128K', 'provider